In [1]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 59 kB 6.2 MB/s 
     |████████████████████████████████| 3.3 MB 32.2 MB/s 
     |████████████████████████████████| 895 kB 62.1 MB/s 
     |████████████████████████████████| 596 kB 56.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('/content/output.csv',index_col=0)
len(data['pagedata'][0])

1885

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')



Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [12]:
import numpy as np
def get_mnli_response(document,question):
    result = {}

    premise = document
    hypothesis = question

    # run through model pre-trained on MNLI
    x = tokenizer.encode(premise, hypothesis, return_tensors='pt')
    logits = nli_model(x.to(torch_device))[0]

    # output = tokenizer.decode(logits)
    labels = ['contradiction','entailment']

    # we throw away "neutral" (dim 1) and take the probability of
    # "entailment" (2) as the probability of the label being true 
    entail_contradiction_logits = logits[:,[0,2]]
    probs = entail_contradiction_logits.softmax(dim=1)
    prob_label_is_true = probs[:,1]
    label = labels[np.argmax(probs[0].tolist())]
    result['label'] = label
    result['mnli_score'] = max(probs[0].tolist())
    return result


In [ ]:
#Code


In [ ]:
import torch
import numpy as np
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from transformers import BartTokenizer, BartForConditionalGeneration
# from transformers import RobertaTokenizer,RobertaForQuestionAnswering
from transformers import AutoTokenizer, AutoModelForQuestionAnswering,AutoModelForSequenceClassification
torch_device = 'cpu'

USE_SUMMARY = False

#---?
QA_MODEL = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad') # where are the models
QA_TOKENIZER = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
MNLI_MODEL = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
MNLI_TOKENIZER =   AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

QA_MODEL.to(torch_device)
QA_MODEL.eval()



if USE_SUMMARY:
    SUMMARY_TOKENIZER = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    SUMMARY_MODEL = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    SUMMARY_MODEL.to(torch_device)
    SUMMARY_MODEL.eval()

## Data Pre Processing

In [6]:
from bs4 import BeautifulSoup
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import re

from tqdm import tqdm
text = []
urls = []
# tqdm is for printing the status bar
for sentance in tqdm(data['pagedata'].values):
    # print('Actual: ' ,sentance)
    # find URLS
    url =  re.findall(r'(https?://\S+)', sentance)
    
    header = ''.join(sentance.split('\n')[0])
    print(header)
    #take the context body
    sentance = ' '.join(sentance.split('\n')[1:-2])
    
    #  
    sentance = re.sub('[^A-Za-z0-9-%.]+', ' ', sentance) 

    #remove URLS
    sentance = re.sub(r"http\S+", "", sentance)
    
    
    
    
    text.append(sentance.strip())
    urls.append(url)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


100%|██████████| 25/25 [00:00<00:00, 3540.81it/s]

11/17/21, 6:12 PM Predictive Analytics Market Size & Trends Report, 2019-2026
11/17/21, 6:12 PM Predictive Analytics Market Size & Trends Report, 2019-2026
11/17/21, 6:12 PM Predictive Analytics Market Size & Trends Report, 2019-2026
11/17/21, 6:12 PM Predictive Analytics Market Size & Trends Report, 2019-2026
11/17/21, 6:12 PM Predictive Analytics Market Size & Trends Report, 2019-2026
11/17/21, 6:12 PM Predictive Analytics Market Size & Trends Report, 2019-2026
11/17/21, 6:12 PM Predictive Analytics Market Size & Trends Report, 2019-2026
11/17/21, 6:12 PM Predictive Analytics Market Size & Trends Report, 2019-2026
11/17/21, 6:07 PM Data Analytics Market Size, Share, Trends, Global and Forecast 2027 | MRFR
11/17/21, 6:07 PM Data Analytics Market Size, Share, Trends, Global and Forecast 2027 | MRFR
11/17/21, 6:07 PM Data Analytics Market Size, Share, Trends, Global and Forecast 2027 | MRFR
11/17/21, 6:07 PM Data Analytics Market Size, Share, Trends, Global and Forecast 2027 | MRFR
11/1

In [7]:
data['text'] = text
data['url'] = urls
data.head()

,id,pagedata,text,url
0,trendreport0,"11/17/21, 6:12 PM Predictive Analytics Market ...",Home Analytics Predictive Analytics Market Pre...,[https://www.reportsanddata.com/report-detail/...
1,trendreport1,"11/17/21, 6:12 PM Predictive Analytics Market ...",Request For A Free Extract Download Summary Th...,[https://www.reportsanddata.com/report-detail/...
2,trendreport2,"11/17/21, 6:12 PM Predictive Analytics Market ...",Company profiling with detailed strategies fin...,[https://www.reportsanddata.com/report-detail/...
3,trendreport3,"11/17/21, 6:12 PM Predictive Analytics Market ...",Among the deployment modes the cloud accounted...,[https://www.reportsanddata.com/report-detail/...
4,trendreport4,"11/17/21, 6:12 PM Predictive Analytics Market ...",Solution Services Deployment Mode Outlook Reve...,[https://www.reportsanddata.com/report-detail/...


In [ ]:
for index,row in data.iterrows():
    print(len(row['text']))
    print('Actual Text: ', row['pagedata'])
    print('Clean Text: ',row['text'] , '\n\n')

1572
1730
2431
3360
771
758
78
446
1631
997
3848
2584
2148
557
747
171
1843
3558
2386
3306
2025
1181
1563
881
939


In [ ]:
import numpy as np
def reconstructText(tokens, start=0, stop=-1):
    tokens = tokens[start: stop]
    if '[SEP]' in tokens:
        sepind = tokens.index('[SEP]')
        tokens = tokens[sepind+1:]
    txt = ' '.join(tokens)
    txt = txt.replace(' ##', '')
    txt = txt.replace('##', '')
    txt = txt.strip()
    txt = " ".join(txt.split())
    txt = txt.replace(' .', '.')
    txt = txt.replace(' . ', '.')
    txt = txt.replace('( ', '(')
    txt = txt.replace(' )', ')')
    txt = txt.replace(' - ', '-')
    txt_list = txt.split(' , ')
    txt = ''
    nTxtL = len(txt_list)
    if nTxtL == 1:
        return txt_list[0]
    newList =[]
    for i,t in enumerate(txt_list):
        if i < nTxtL -1:
            if t[-1].isdigit() and txt_list[i+1][0].isdigit():
                newList += [t,',']
            else:
                newList += [t, ', ']
        else:
            newList += [t]
    return ''.join(newList)


def makeBERTSQuADPrediction(document, question):
    ## we need to rewrite this function so that it chuncks the document into 250-300 word segments with
    ## 50 word overlaps on either end so that it can understand and check longer abstracts
    nWords = len(document.split())
    input_ids_all = QA_TOKENIZER.encode(question, document)
    tokens_all = QA_TOKENIZER.convert_ids_to_tokens(input_ids_all)
    overlapFac = 1.1
    # print(input_ids_all)

    if len(input_ids_all)*overlapFac > 2048:
        nSearchWords = int(np.ceil(nWords/5))
        quarter = int(np.ceil(nWords/4))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[quarter-int(nSearchWords*overlapFac/2):quarter+int(quarter*overlapFac/2)]),
                     ' '.join(docSplit[quarter*2-int(nSearchWords*overlapFac/2):quarter*2+int(quarter*overlapFac/2)]),
                     ' '.join(docSplit[quarter*3-int(nSearchWords*overlapFac/2):quarter*3+int(quarter*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
        
    elif len(input_ids_all)*overlapFac > 1536:
        nSearchWords = int(np.ceil(nWords/4))
        third = int(np.ceil(nWords/3))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[third-int(nSearchWords*overlapFac/2):third+int(nSearchWords*overlapFac/2)]),
                     ' '.join(docSplit[third*2-int(nSearchWords*overlapFac/2):third*2+int(nSearchWords*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
        
    elif len(input_ids_all)*overlapFac > 1024:
        nSearchWords = int(np.ceil(nWords/3))
        middle = int(np.ceil(nWords/2))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[middle-int(nSearchWords*overlapFac/2):middle+int(nSearchWords*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
    
    elif len(input_ids_all)*overlapFac > 512:
        nSearchWords = int(np.ceil(nWords/2))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
    else:
        input_ids = [input_ids_all]
    absTooLong = False    
    
    answers = []
    cons = []

    for iptIds in input_ids:
        tokens = QA_TOKENIZER.convert_ids_to_tokens(iptIds)
        sep_index = iptIds.index(QA_TOKENIZER.sep_token_id)
        num_seg_a = sep_index + 1
        num_seg_b = len(iptIds) - num_seg_a
        segment_ids = [0]*num_seg_a + [1]*num_seg_b
        assert len(segment_ids) == len(iptIds)
        n_ids = len(segment_ids)
        #print(n_ids)

        if n_ids < 512:
            res = QA_MODEL(torch.tensor([iptIds]).to(torch_device), 
                                     token_type_ids=torch.tensor([segment_ids]).to(torch_device))
            
            # print(res[0][:,1:-1])
            # print(res.start_logits)
            # start_scores, end_scores = QA_MODEL(torch.tensor([iptIds]).to(torch_device), 
            #                          token_type_ids=torch.tensor([segment_ids]).to(torch_device))
            start_scores = res.start_logits
            end_scores = res.end_logits

            # print(start_scores.logits)

        else:
            
            #this cuts off the text if its more than 512 words so it fits in model space
            #need run multiple inferences for longer text. add to the todo
            print('****** warning only considering first 512 tokens, document is '+str(nWords)+' words long.  There are '+str(n_ids)+ ' tokens')
            absTooLong = True
            res = QA_MODEL(torch.tensor([iptIds[:512]]).to(torch_device), 
                                     token_type_ids=torch.tensor([segment_ids[:512]]).to(torch_device))
            start_scores = res.start_logits
            end_scores = res.end_logits
            
        # print(start_scores.logits)

        start_scores = start_scores[:,1:-1]
        end_scores = end_scores[:,1:-1]
        answer_start = torch.argmax(start_scores)
        answer_end = torch.argmax(end_scores)
        #print(answer_start, answer_end)
        answer = reconstructText(tokens, answer_start, answer_end+5)
    
        if answer.startswith('.') or answer.startswith(','):
            answer = answer[2:]
            
        c = start_scores[0,answer_start].item()+end_scores[0,answer_end].item()
        answers.append(answer)
        cons.append(c)
    
    maxC = max(cons)
    iMaxC = [i for i, j in enumerate(cons) if j == maxC][0]
    confidence = cons[iMaxC]
    answer = answers[iMaxC]
    
    sep_index = tokens_all.index('[SEP]')
    full_txt_tokens = tokens_all[sep_index+1:]
    
    abs_returned = reconstructText(full_txt_tokens)

    ans={}
    ans['answer'] = answer
    #print(answer)
    if answer.startswith('[CLS]') or answer_end.item() < sep_index or answer.endswith('[SEP]'):
        ans['confidence'] = -1000000
    else:
        #confidence = torch.max(start_scores) + torch.max(end_scores)
        #confidence = np.log(confidence.item())
        ans['confidence'] = confidence
    #ans['start'] = answer_start.item()
    #ans['end'] = answer_end.item()
    ans['abstract_bert'] = abs_returned
    ans['abs_too_long'] = absTooLong
    return ans

In [ ]:
question = 'what is advanced analytics market growth?'
documents = data['text']


for each in documents[:10] :
    print(each)
    answer = makeBERTSQuADPrediction(each, question)
    print(answer,'\n\n')

Home Analytics Predictive Analytics Market Predictive Analytics Market Size Share And Industry Analysis By End Use Sales and Marketing Management Financial Management Customer Behavior Analysis Supply Chain Management Network Traffic Management Demand And Supply Forecasting Risk Management and Fraud Detection Others By Component Solution Services By Organization Size Small and Medium-Sized Enterprise Large Enterprises By Deployment Mode Cloud and On-premises By Industry Vertical Government and Defense Healthcare and Life Sciences Manufacturing Retail and E-commerce IT and Telecommunication Energy and Utilities Banking Financial Services and Insurance BFSI Others And Region Segment Forecasts To 2026 Category Information and Communication ID Format Published Date Jan Pages Technology RND 002320 PDF 2020 160 Our team of researchers are studying Covid19 and its impact on various industry verticals a Report Description Table of Contents Research Methodology Market Summary The global predict

Token indices sequence length is longer than the specified maximum sequence length for this model (603 > 512). Running this sequence through the model will result in indexing errors


{'answer': 'company profiling', 'confidence': -1000000, 'abstract_bert': 'company profiling with detailed strategies financials and recent developments. request for a free extract download summary salient trends of the predictive analytics market the popularity of data mining is owing to its ability to collect and analyze large amounts of data to find redundant patterns. the data is structured to discover useful trends to harness future information. artificial intelligence is another trend that combines historical algorithms with programming. it can be used to find trends and patterns in the data for further use. the artificial intelligence is combined with machine learning through algorithms to identify patterns within millions of data units and predict the behaviors to make future decisions without any human intervention. deep learning is made up of an artificial neural network with several hierarchical levels. the information learned in each level is carried over into the next level

In [ ]:
question = 'Which enterprise size segment led the advanced analytics market? '

for each in documents[:10] :
    print(each)
    answer = makeBERTSQuADPrediction(each, question)
    print(answer,'\n\n')

Home Analytics Predictive Analytics Market Predictive Analytics Market Size Share And Industry Analysis By End Use Sales and Marketing Management Financial Management Customer Behavior Analysis Supply Chain Management Network Traffic Management Demand And Supply Forecasting Risk Management and Fraud Detection Others By Component Solution Services By Organization Size Small and Medium-Sized Enterprise Large Enterprises By Deployment Mode Cloud and On-premises By Industry Vertical Government and Defense Healthcare and Life Sciences Manufacturing Retail and E-commerce IT and Telecommunication Energy and Utilities Banking Financial Services and Insurance BFSI Others And Region Segment Forecasts To 2026 Category Information and Communication ID Format Published Date Jan Pages Technology RND 002320 PDF 2020 160 Our team of researchers are studying Covid19 and its impact on various industry verticals a Report Description Table of Contents Research Methodology Market Summary The global predict

In [14]:
import torch 
question = "How is the advanced analytics market is expected to grow ?"
torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


for each in data['text'] :
    print(each)
    answer = get_mnli_response(each, question)
    print(answer,'\n\n')

Home Analytics Predictive Analytics Market Predictive Analytics Market Size Share And Industry Analysis By End Use Sales and Marketing Management Financial Management Customer Behavior Analysis Supply Chain Management Network Traffic Management Demand And Supply Forecasting Risk Management and Fraud Detection Others By Component Solution Services By Organization Size Small and Medium-Sized Enterprise Large Enterprises By Deployment Mode Cloud and On-premises By Industry Vertical Government and Defense Healthcare and Life Sciences Manufacturing Retail and E-commerce IT and Telecommunication Energy and Utilities Banking Financial Services and Insurance BFSI Others And Region Segment Forecasts To 2026 Category Information and Communication ID Format Published Date Jan Pages Technology RND 002320 PDF 2020 160 Our team of researchers are studying Covid19 and its impact on various industry verticals a Report Description Table of Contents Research Methodology Market Summary The global predict

## Indexing Code and Retrival Code

In [20]:
import urllib.request
request_url = urllib.request.urlopen('http://127.0.0.1:5000/searching/getdocs/consumer')
# print(request_url.read())

URLError: ignored

In [7]:
from urllib.request import urlopen
import json


query = 'what%20is%20the%20revenue%20of%20consumer%20industry%3F'

connection = urlopen('http://127.0.0.1:5000/searching/getdocs/consumer') 

# response = json.load(connection)

URLError: ignored

In [13]:
!pip install elasticsearch

     |████████████████████████████████| 379 kB 4.1 MB/s 


In [14]:
from elasticsearch import Elasticsearch
# importing required modules
# import PyPDF2,pdfplumber
# import pandas as pd
# import numpy as np
# import logging as lg,json,re


elasticserverHost='localhost'
pdftoload='.\\traindata\\trendreport.pdf'
pdftoload1='.\\traindata\\shareforecast.pdf'
pdftoload2='.\\traindata\\marketanalysis.pdf'


class ElasticServerActivityManager(object):

    def __init__(self):
        self.elastic_server = Elasticsearch(hosts=elasticserverHost,port=9200)
        self.server_host=elasticserverHost

    def getElasticServerReference(self):
        if self.elastic_server.ping():
            return self.elastic_server
        else:
            raise Exception(f'Unable to connect elastic search server at {elasticserverHost}:{9200}')
    
    # def saveDatainIndex(self,index,key,data):
    #     try:
    #         return self.elastic_server.index(index=index,id=key,document=data)
    #     except Exception as ex:
    #         raise Exception(f"Error saving data in index {index}: {ex}")
    
    # def createNewIndex(self,indexName,mapping=None):
    #     # mapping = {
    #     #     # "mappings": {
    #     #         "properties": {
    #     #             "values": {
    #     #                 "type": "binary"
    #     #             }
    #     #         }
    #     #     # }
    #     # }
    #     try:
    #         if not self.elastic_server.indices.exists(index=indexName):
    #             if mapping:
    #                 self.elastic_server.indices.create(index=indexName,mappings=mapping)
    #             else:
    #                 self.elastic_server.indices.create(index=indexName)
    #         else:
    #             print('its already there ')
    #         return True
    #     except Exception as ex:
    #         raise Exception(f"Error creating new index {indexName}: {ex}")

    # def deleteAllDocFromIndex(self,indexName):
    #     return self.elastic_server.delete_by_query(index=indexName,body={
    #                                                             "query": {
    #                                                                 "match_all": {}
    #                                                             }
    #                                                         })

    def searchKeyword(self,searchText=None,suggestionrequest=False):
        try:
            queryString={
                "query":{
                    "bool":{
                        "should":[{
                            "wildcard": {
                                "values": {
                                    "value": f"*{searchText}*"
                                }
                            }
                        },
                        {
                            "match":{
                                "values":{
                                    "query":f"{searchText}"
                                }
                            }
                        },{
                            "wildcard": {
                                "Opportunity Name": {
                                    "value": f"*{searchText}*"
                                }
                            }
                        },
                        {
                            "match":{
                                "Opportunity Name":{
                                    "query":f"{searchText}"
                                }
                            }
                        },{
                            "wildcard": {
                                "question": {
                                    "value": f"*{searchText}*"
                                }
                            }
                        },
                        {
                            "match":{
                                "question":{
                                    "query":f"{searchText}"
                                }
                            }
                        }]
                    }
                }
            }
            if suggestionrequest:
                results_search=self.elastic_server.search(index=['questions'],body=json.dumps(queryString),_source=True)['hits']['hits']
            else:
                results_search=self.elastic_server.search(index=['marketreport','sfdcreport'],body=json.dumps(queryString),_source=True)['hits']['hits']
            
            if results_search:
                resultset=[]
                for data in  results_search:
                    content=data["_source"]
                    content.update({'confidence':data['_score']})
                    resultset.append(content)
                return resultset
            else:
                raise Exception(f"No data found against search text {searchText}")
            
        except Exception as ex:
            raise Exception(f'{ex}')



In [15]:

elastic_server = ElasticServerActivityManager()


In [17]:
elastic_server.searchKeyword(searchText = 'revenue')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:113: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information


Exception: ignored

## Additional Questions

In [ ]:
question_list = ['What are the factors driving the advanced analytics market growth?',
'How big is the advanced analytics market?',
'What is advanced analytics market growth?',
'Which type segment accounted for the highest advanced analytics market share?',
'Who are the key players in the advanced analytics market?',
'Which deployment segment held the largest advanced analytics market share?', 
'Which enterprise size segment led the advanced analytics market?', 
'Which end-use segment dominated the advanced analytics market?', 
'Which region held the highest advanced analytics market share?']

## SFDC DATA

In [ ]:
bu_data = pd.read_excel('/content/Sanitized sfdc data.xlsx')
bu_data['Created Date'] = pd.to_datetime(bu_data['Created Date'])
bu_data['Last Stage Change Date'] = pd.to_datetime(bu_data['Last Stage Change Date'])
bu_data

,Account Name,Opportunity Name,Opportunity Owner,P&L BU,Account Type,Stage,Bi2i Industry,Bi2i Sub-Industry,Created Date,Last Stage Change Date,Probability (%),Sum of Amount,Sum of Expected Revenue
0,Three Compassion,Three Compassion Renewal for FY 2021 - Part1 Q1,Suvodip Chatterjee,Enterprise Tech,B,Closed Won,Consumer,B2C Tech,2019-03-12,2020-04-15,100,53571.0,53571.0
1,Three Compassion,Three Compassion Renewal for FY 2021 - Part2 Q1,Suvodip Chatterjee,Enterprise Tech,B,Closed Won,Consumer,B2C Tech,2020-03-23,2020-04-15,100,21426.0,21426.0
2,Three Compassion,Three Compassion - FARE Web App Project,Suvodip Chatterjee,Enterprise Tech,B,Closed Won,Consumer,B2C Tech,2019-11-28,2020-03-03,100,14286.0,14286.0
3,Three Compassion,Three Compassion 2nd Admin contractor Jan20-Ju...,Suvodip Chatterjee,Enterprise Tech,B,Closed Won,Consumer,B2C Tech,2020-01-21,2020-03-03,100,2624.0,2624.0
4,Three Compassion,Three Compassion Admin contractor,Suvodip Chatterjee,Enterprise Tech,B,Closed Won,Consumer,B2C Tech,2019-06-20,2019-05-11,100,3960.0,3960.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,Sigmasocial,Sigmasocial- R&D R Shiny,Sonal Gupta,Consumer,A,Closed Won,Consumer,CPG,2020-02-17,2020-12-10,100,50000.0,50000.0
283,Sigmasocial,Sigmasocial Supply Chain- S&OP POC,Sonal Gupta,Consumer,A,Closed Won,Consumer,CPG,2018-05-23,2019-01-30,100,40000.0,40000.0
284,Sigmasocial,Sigmasocial-Sales Force Route Optimization,Sonal Gupta,Consumer,A,Closed Won,Consumer,CPG,2019-08-22,2019-11-18,100,32164.0,32164.0
285,Sigmasocial,Sigmasocial-Sales Force Route Optimization Pha...,Sonal Gupta,Consumer,A,Closed Won,Consumer,CPG,2019-05-11,2019-12-30,100,126786.0,126786.0


In [ ]:


text_data = []
for index,row in bu_data.iterrows():
    txt = 'Account Name '+ row['Account Name'] + ' Opportunity Name '+ row['Opportunity Name'] + ' P&L BU '+ row['P&L BU'] + ' Account Type '+ row['Account Type'] + ' Bi2i Industry '+ row['Bi2i Industry'] + ' Bi2i Sub-Industry '+ row['Bi2i Sub-Industry'] + ' Sum of Amount '+ str(row['Sum of Amount']) + ' Sum of Expected Revenue '+ str(row['Sum of Expected Revenue'])
    # print('context: ', txt)
    text_data.append(txt)

bu_data['text'] = text_data

In [ ]:
question = 'what is the net revenue of consumer industry?'



for each in text_data[:4] :
    print('Q :',question)
    print('context:',each)
    answer = makeBERTSQuADPrediction(each, question)
    print('A:',answer,'\n\n')

Q : what is the net revenue of consumer industry?
context: Account Name Three Compassion Opportunity Name Three Compassion Renewal for FY 2021 - Part1 Q1 P&L BU Enterprise Tech Account Type B Bi2i Industry Consumer Bi2i Sub-Industry B2C Tech Sum of Amount 53571.0 Sum of Expected Revenue 53571.0
A: {'answer': 'amount 53571. 0', 'confidence': 6.992535591125488, 'abstract_bert': 'account name three compassion opportunity name three compassion renewal for fy 2021-part1 q1 p & l bu enterprise tech account type b bi2i industry consumer bi2i sub-industry b2c tech sum of amount 53571. 0 sum of expected revenue 53571. 0', 'abs_too_long': False} 


Q : what is the net revenue of consumer industry?
context: Account Name Three Compassion Opportunity Name Three Compassion Renewal for FY 2021 - Part2 Q1 P&L BU Enterprise Tech Account Type B Bi2i Industry Consumer Bi2i Sub-Industry B2C Tech Sum of Amount 21426.0 Sum of Expected Revenue 21426.0
A: {'answer': 'amount 21426. 0', 'confidence': 7.23621606

In [ ]:
sample_text = "".join(text_data[:3])
sample_text

'Account Name Three Compassion Opportunity Name Three Compassion Renewal for FY 2021 - Part1 Q1 P&L BU Enterprise Tech Account Type B Bi2i Industry Consumer Bi2i Sub-Industry B2C Tech Sum of Amount 53571.0 Sum of Expected Revenue 53571.0Account Name Three Compassion Opportunity Name Three Compassion Renewal for FY 2021 - Part2 Q1 P&L BU Enterprise Tech Account Type B Bi2i Industry Consumer Bi2i Sub-Industry B2C Tech Sum of Amount 21426.0 Sum of Expected Revenue 21426.0Account Name Three Compassion Opportunity Name Three Compassion - FARE Web App Project P&L BU Enterprise Tech Account Type B Bi2i Industry Consumer Bi2i Sub-Industry B2C Tech Sum of Amount 14286.0 Sum of Expected Revenue 14286.0'

In [ ]:
answer = makeBERTSQuADPrediction(sample_text, question)
print('A:',answer,'\n\n')

A: {'answer': 'industry b2c tech', 'confidence': 1.049033135175705, 'abstract_bert': 'account name three compassion opportunity name three compassion renewal for fy 2021-part1 q1 p & l bu enterprise tech account type b bi2i industry consumer bi2i sub-industry b2c tech sum of amount 53571. 0 sum of expected revenue 53571. 0account name three compassion opportunity name three compassion renewal for fy 2021-part2 q1 p & l bu enterprise tech account type b bi2i industry consumer bi2i sub-industry b2c tech sum of amount 21426. 0 sum of expected revenue 21426. 0account name three compassion opportunity name three compassion-fare web app project p & l bu enterprise tech account type b bi2i industry consumer bi2i sub-industry b2c tech sum of amount 14286. 0 sum of expected revenue 14286. 0', 'abs_too_long': False} 




Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

ImportError: ignored